<a href="https://colab.research.google.com/github/Everysimo/Muxi_SE4AI/blob/main/Muxi_BERT_NamedEntityRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Data Extraction

In [1]:
!pip install simpletransformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 77.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 71.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 63.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.7/214.7 kB 25.9 MB/s eta 0:00:00


In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import os
from six.moves import urllib
import torch

if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

plt.style.use("seaborn-v0_8-whitegrid")

plt.rcParams['figure.figsize'] = (20,10)

DOWNLOAD_ROOT="https://raw.githubusercontent.com/Everysimo/Muxi_SE4AI/main/NERCambiaScenarioDataSet.csv"

#fetch dataset da github
def fetch_file_data1(file_url1=DOWNLOAD_ROOT, file_path1="file"):
  os.makedirs(file_path1, exist_ok=True)
  csv_path1 = os.path.join(file_path1, "IntentsMuxyDataset.csv")
  urllib.request.urlretrieve(file_url1, csv_path1)
fetch_file_data1()

#data è la variabile che contiene il dataset
data=pd.read_csv("file/IntentsMuxyDataset.csv")
#data=df[["Intent", "Text"]]
data

,Sentence #,Word,Tag
0,Frase1,Cambia,O
1,Frase1,l'atmosfera,O
2,Frase1,della,O
3,Frase1,presentazione,B_NOME_SCENARIO
4,Frase1,.,O
...,...,...,...
1813,Frase220,l'ambientazione,O
1814,Frase220,per,O
1815,Frase220,una,O
1816,Frase220,presentazione,O


In [3]:
data.head(30)

,Sentence #,Word,Tag
0,Frase1,Cambia,O
1,Frase1,l'atmosfera,O
2,Frase1,della,O
3,Frase1,presentazione,B_NOME_SCENARIO
4,Frase1,.,O
5,Frase2,Puoi,O
6,Frase2,modificare,O
7,Frase2,l'ambiente,O
8,Frase2,per,O
9,Frase2,una,O


In [4]:
data =data.fillna(method ="ffill")

In [5]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [6]:
data.rename(columns={"Sentence #":"sentence_id","Word":"words","Tag":"labels"}, inplace =True)


In [7]:
data["labels"] = data["labels"].str.upper()


In [8]:
X= data[["sentence_id","words"]]
Y =data["labels"]

In [9]:
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size =0.2)

In [10]:
#building up train data and test data
train_data = pd.DataFrame({"sentence_id":x_train["sentence_id"],"words":x_train["words"],"labels":y_train})
test_data = pd.DataFrame({"sentence_id":x_test["sentence_id"],"words":x_test["words"],"labels":y_test})

In [11]:
train_data

,sentence_id,words,labels
1381,Frase165,la,O
73,Frase9,stanza,O
1493,Frase180,Cambia,O
1807,Frase219,contesto,O
1372,Frase164,scenario,O
...,...,...,...
1328,Frase159,il,O
1758,Frase213,uno,O
477,Frase55,online,O
1509,Frase182,scenario,O


##Model Training


In [12]:
from simpletransformers.ner import NERModel,NERArgs
label = data["labels"].unique().tolist()
label

['O', 'B_NOME_SCENARIO']

In [18]:
args = NERArgs()
args.num_train_epochs = 10
args.learning_rate = 1e-4
args.overwrite_output_dir =True
args.train_batch_size = 32
args.eval_batch_size = 32

model = NERModel('bert', 'bert-base-cased')

model.train_model(train_data,eval_data = test_data,acc=accuracy_score)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['cl

/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/1 [00:00<?, ?it/s]

KeyError: ignored

In [ ]:
model = NERModel('bert', 'bert-base-cased',labels=label,args =args)


In [ ]:
model.train_model(train_data,eval_data = test_data,acc=accuracy_score)

In [ ]:
result, model_outputs, preds_list = model.eval_model(test_data)

In [ ]:
result

In [ ]:
prediction, model_output = model.predict(["crea una riunione nominata ciao amici per parlare delle pubbliche relazioni il sette gennaio alle quindici e mezza fino alle sedici"])
prediction

#Save model


In [ ]:
# Su Colab
import torch
import pickle

model.model.save_pretrained('model1')
model.tokenizer.save_pretrained('model1')
model.config.save_pretrained('model1/')